### Script to correlate estimates from cognitive models with brain acitvyt from ROI analysis

In [ ]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'

import pandas as pd
import numpy as np
import shutil
import itertools
import statannot
import scipy

import glob
import re
import seaborn as sns
import statsmodels.api as sm
import statsmodels
import scipy
import nideconv
from nideconv import GroupResponseFitter
from scipy import signal
from scipy.stats.stats import pearsonr

import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.backends.backend_pdf

%matplotlib inline

def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
# some functions to load data
def load_single_msit(): # load PMWG msit model used in paper
    
    cogmod = pd.read_csv(f'scott/single_msit_params.tsv',sep=',')
    cogmod['subject'] = cogmod.index
    cogmod.subject = cogmod.subject.astype(str)
    
    return cogmod

def load_betas(contrast): # load ROI activity
    
    betas = pd.read_csv(f'scott/{contrast}_betas_tsv.tsv',sep='\t')
    betas.subject = betas.subject.astype(str)
    
    return betas

# def load_cogmod(): # joint model not used in paper
    
#     cogmod = pd.read_csv(f'scott/joint_model_params.tsv',sep='\t')
#     cogmod['subject'] = cogmod.index
#     cogmod['SSRT'] = cogmod['SST|muS'] + cogmod['SST|tauS']
#     cogmod.subject = cogmod.subject.astype(str)
    
#     return cogmod

def load_mean_ssrt(): # load SSRT estimates using mean method
    ssrt = pd.read_csv('scott/mean_ssrt.tsv', sep='\t')
    return ssrt

def load_dmc_params(): # load DMC sst model used in paper
    
    dmc = pd.read_csv('scott/dmc_params_final.tsv', sep='\t') 
    dmc.drop(dmc.tail(1).index,inplace=True) # drop last n rows dmc.reset_index() 
    dmc['SSRT'] = dmc['muS'] + dmc['tauS']
    return dmc

# SST

First correlate SSRT calcuated using BEESTS with SS, SS > GO, FS > SS 

Second, correlate SSRT calcuated using the mean method with SS, SS > GO, FS > SS 



# MEAN METHOD

### SSRT (mean method) vs SS

In [ ]:
betas = load_betas('ss')
cog = load_mean_ssrt()
cog.columns = ['subject','SSRT']
cog.subject = cog.subject.astype(str)

all_subs = sorted(list(set(betas.subject).intersection(cog.subject)),key=int)
print(len(all_subs)) # there should be 31

cog = cog.loc[cog.subject.isin(all_subs)].reset_index(drop=True)
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

cog_betas = pd.concat([cog, betas], axis=1)
cog_betas = cog_betas.loc[:,~cog_betas.columns.duplicated()]


# corr = cog_betas.corr()
# corr.style.background_gradient(cmap='coolwarm')

for i in cog_betas.columns:
    if i == 'subject':
        continue
    r, p = pearsonr(cog_betas['SSRT'], cog_betas[i])
    if p <= 0.05:
        print(f'{i}\n r = {r}\n p = {p}')
        
        ## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = cog_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=cog_betas['SSRT'], y=cog_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi: 
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.590),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.430),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

### SSRT (mean method) vs ss>go

In [ ]:
betas_ss = load_betas('ss')
betas_go = load_betas('go')
betas_ss.iloc[:,4:] = betas_ss.iloc[:,4:] - betas_go.iloc[:,4:]
betas = betas_ss

cog = load_mean_ssrt()
cog.columns = ['subject','SSRT']
cog.subject = cog.subject.astype(str)

all_subs = sorted(list(set(betas.subject).intersection(cog.subject)),key=int)

cog = cog.loc[cog.subject.isin(all_subs)].reset_index(drop=True)
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

cog_betas = pd.concat([cog, betas], axis=1)
cog_betas = cog_betas.loc[:,~cog_betas.columns.duplicated()]

for i in cog_betas.columns:
    if i == 'subject':
        continue
    r, p = pearsonr(cog_betas['SSRT'], cog_betas[i])
    if p <= 0.05:
        print(f'{i}\n r = {r}\n p = {p}')
        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = cog_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=cog_betas['SSRT'], y=cog_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (SS > Go)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi: 
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.45),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.345),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

### SSRT (mean method) vs fs>ss

In [ ]:
betas_fs = load_betas('fs')
betas_ss = load_betas('ss')
betas_fs.iloc[:,4:] = betas_fs.iloc[:,4:] - betas_ss.iloc[:,4:]
betas = betas_fs

cog = load_mean_ssrt()
cog.columns = ['subject','SSRT']
cog.subject = cog.subject.astype(str)

all_subs = sorted(list(set(betas.subject).intersection(cog.subject)),key=int)

cog = cog.loc[cog.subject.isin(all_subs)].reset_index(drop=True)
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

cog_betas = pd.concat([cog, betas], axis=1)
cog_betas = cog_betas.loc[:,~cog_betas.columns.duplicated()]

for i in cog_betas.columns:
    if i == 'subject':
        continue
    r, p = pearsonr(cog_betas['SSRT'], cog_betas[i])
    if p <= 0.05:
        print(f'{i}\n r = {r}\n p = {p}')
        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = cog_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=cog_betas['SSRT'], y=cog_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (FS > SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi: 
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.145,0.775),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.145,0.605),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

# BEESTS METHOD

### SSRT (DMC) vs SS

In [ ]:
betas = load_betas('ss') 
# dmc = pd.read_csv('scott/dmc_params_final.tsv', sep='\t') 
# dmc.drop(dmc.tail(1).index,inplace=True) # drop last n rows dmc.reset_index() 
# dmc['SSRT'] = dmc['muS'] + dmc['tauS']

dmc = load_dmc_params()

all_subs = sorted(list(set(betas.subject).intersection(dmc.index)),key=int)

dmc = dmc.loc[dmc.index.isin(all_subs)].reset_index(drop=True) 
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

dmc_betas = pd.concat([dmc, betas], axis=1) 
dmc_betas = dmc_betas.loc[:,~dmc_betas.columns.duplicated()]

dmc_betas = dmc_betas.drop(['event type','regressor','covariate'],axis=1)

# show correlations
for i in dmc_betas.columns:
    if i == 'subject':
        continue
    r, p = pearsonr(dmc_betas['SSRT'], dmc_betas[i])
    if p <= 0.05:
        print(f'{i}\n r = {r}\n p = {p}')
        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = dmc_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=dmc_betas['SSRT'], y=dmc_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.3,0.555),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.3,0.415),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

### SSRT (DMC) vs SS > Go

In [ ]:
betas_ss = load_betas('ss')
betas_go = load_betas('go')
betas_ss.iloc[:,4:] = betas_ss.iloc[:,4:] - betas_go.iloc[:,4:]
betas = betas_ss
# dmc = pd.read_csv('scott/dmc_params_final.tsv', sep='\t') 
# dmc.drop(dmc.tail(1).index,inplace=True) # drop last n rows dmc.reset_index() 
# dmc['SSRT'] = dmc['muS'] + dmc['tauS']

dmc = load_dmc_params()

all_subs = sorted(list(set(betas.subject).intersection(dmc.index)),key=int)

dmc = dmc.loc[dmc.index.isin(all_subs)].reset_index(drop=True) 
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

dmc_betas = pd.concat([dmc, betas], axis=1) 
dmc_betas = dmc_betas.loc[:,~dmc_betas.columns.duplicated()]

dmc_betas = dmc_betas.drop(['event type','regressor','covariate'],axis=1)

# show correlations
for i in dmc_betas.columns:
    if i == 'subject':
        continue
    r, p = pearsonr(dmc_betas['SSRT'], dmc_betas[i])
    if p <= 0.05:
        print(f'{i}\n r = {r}\n p = {p}')
        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = dmc_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=dmc_betas['SSRT'], y=dmc_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (SS > GO)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.3,0.440),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.3,0.335),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

### SSRT (DMC) vs FS > SS

In [ ]:
betas_fs = load_betas('fs')
betas_ss = load_betas('ss')
betas_fs.iloc[:,4:] = betas_fs.iloc[:,4:] - betas_ss.iloc[:,4:]
betas = betas_fs
# dmc = pd.read_csv('scott/dmc_params_final.tsv', sep='\t') 
# dmc.drop(dmc.tail(1).index,inplace=True) # drop last n rows dmc.reset_index() 
# dmc['SSRT'] = dmc['muS'] + dmc['tauS']

dmc = load_dmc_params()

all_subs = sorted(list(set(betas.subject).intersection(dmc.index)),key=int)

dmc = dmc.loc[dmc.index.isin(all_subs)].reset_index(drop=True) 
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

dmc_betas = pd.concat([dmc, betas], axis=1) 
dmc_betas = dmc_betas.loc[:,~dmc_betas.columns.duplicated()]

dmc_betas = dmc_betas.drop(['event type','regressor','covariate'],axis=1)

# show correlations
for i in dmc_betas.columns:
    if i == 'subject':
        continue
    r, p = pearsonr(dmc_betas['SSRT'], dmc_betas[i])
    if p <= 0.05:
        print(f'{i}\n r = {r}\n p = {p}')
        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = dmc_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=dmc_betas['SSRT'], y=dmc_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (FS > SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.2,0.775),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.2,0.605),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

# Put all SST supplementary material graphs together

### meen method

In [ ]:
betas = load_betas('ss')
cog = load_mean_ssrt()
cog.columns = ['subject','SSRT']
cog.subject = cog.subject.astype(str)

all_subs = sorted(list(set(betas.subject).intersection(cog.subject)),key=int)

cog = cog.loc[cog.subject.isin(all_subs)].reset_index(drop=True)
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

cog_betas = pd.concat([cog, betas], axis=1)
cog_betas = cog_betas.loc[:,~cog_betas.columns.duplicated()]

        
        ## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = cog_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)


# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=cog_betas['SSRT'], y=cog_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi: 
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.590),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.430),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

# ---------------------------------------------

betas_ss = load_betas('ss')
betas_go = load_betas('go')
betas_ss.iloc[:,4:] = betas_ss.iloc[:,4:] - betas_go.iloc[:,4:]
betas = betas_ss

cog = load_mean_ssrt()
cog.columns = ['subject','SSRT']
cog.subject = cog.subject.astype(str)

all_subs = sorted(list(set(betas.subject).intersection(cog.subject)),key=int)

cog = cog.loc[cog.subject.isin(all_subs)].reset_index(drop=True)
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

cog_betas = pd.concat([cog, betas], axis=1)
cog_betas = cog_betas.loc[:,~cog_betas.columns.duplicated()]

        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = cog_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

# print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=cog_betas['SSRT'], y=cog_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (SS > Go)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi: 
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.45),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.33,0.345),fontsize=18,color='cornflowerblue')

# ---------------------------------------------

betas_fs = load_betas('fs')
betas_ss = load_betas('ss')
betas_fs.iloc[:,4:] = betas_fs.iloc[:,4:] - betas_ss.iloc[:,4:]
betas = betas_fs

cog = load_mean_ssrt()
cog.columns = ['subject','SSRT']
cog.subject = cog.subject.astype(str)

all_subs = sorted(list(set(betas.subject).intersection(cog.subject)),key=int)

cog = cog.loc[cog.subject.isin(all_subs)].reset_index(drop=True)
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

cog_betas = pd.concat([cog, betas], axis=1)
cog_betas = cog_betas.loc[:,~cog_betas.columns.duplicated()]
        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = cog_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

# print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=cog_betas['SSRT'], y=cog_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (FS > SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi: 
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.145,0.775),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_betas["SSRT"], cog_betas[this_roi])[0].round(3)}', (0.145,0.605),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))



### BEESSTS method

In [ ]:
betas = load_betas('ss') 
# dmc = pd.read_csv('scott/dmc_params_final.tsv', sep='\t') 
# dmc.drop(dmc.tail(1).index,inplace=True) # drop last n rows dmc.reset_index() 
# dmc['SSRT'] = dmc['muS'] + dmc['tauS']

dmc = load_dmc_params()

all_subs = sorted(list(set(betas.subject).intersection(dmc.index)),key=int)

dmc = dmc.loc[dmc.index.isin(all_subs)].reset_index(drop=True) 
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

dmc_betas = pd.concat([dmc, betas], axis=1) 
dmc_betas = dmc_betas.loc[:,~dmc_betas.columns.duplicated()]

dmc_betas = dmc_betas.drop(['event type','regressor','covariate'],axis=1)

        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = dmc_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

# print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=dmc_betas['SSRT'], y=dmc_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.3,0.555),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.3,0.415),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))

# ---------------------------------------------

betas_fs = load_betas('fs')
betas_ss = load_betas('ss')
betas_fs.iloc[:,4:] = betas_fs.iloc[:,4:] - betas_ss.iloc[:,4:]
betas = betas_fs
# dmc = pd.read_csv('scott/dmc_params_final.tsv', sep='\t') 
# dmc.drop(dmc.tail(1).index,inplace=True) # drop last n rows dmc.reset_index() 
# dmc['SSRT'] = dmc['muS'] + dmc['tauS']

dmc = load_dmc_params()

all_subs = sorted(list(set(betas.subject).intersection(dmc.index)),key=int)

dmc = dmc.loc[dmc.index.isin(all_subs)].reset_index(drop=True) 
betas = betas.loc[betas.subject.isin(all_subs)].reset_index(drop=True)

dmc_betas = pd.concat([dmc, betas], axis=1) 
dmc_betas = dmc_betas.loc[:,~dmc_betas.columns.duplicated()]

dmc_betas = dmc_betas.drop(['event type','regressor','covariate'],axis=1)

        
## correct for multiple comparisons
# pick regions we expect correlations with
# dmc_betas = 
dmc_betas_cor = dmc_betas[['subject','IFG-l','IFG-r','SN-l','SN-r','STN-l','STN-r','SSRT']] # 'IFG-l','IFG-r',

dmc_betas_cor = dmc_betas_cor.set_index('subject')
dmc_betas_cor = dmc_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = dmc_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, dmc_betas_cor.loc[dmc_betas_cor.ROI=='SSRT','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)
stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)

# print(stat_df)

# left anf right hemispehere of ROIs correlated with SSRT
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['SN','STN','IFG']
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=dmc_betas['SSRT'], y=dmc_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_xlabel('SSRT (S)', fontsize=22)
        ax[i].set_title(r,fontsize=26)
        if i == 0:
            ax[i].set_ylabel('beta (FS > SS)',fontsize=24)
        else:
            ax[i].set_ylabel('',fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=21)
        if '-l' in this_roi:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.2,0.775),fontsize=18,color='darkblue')
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(dmc_betas["SSRT"], dmc_betas[this_roi])[0].round(3)}', (0.2,0.605),fontsize=18,color='cornflowerblue')
#     stat_df.loc[stat_df.index == r,'p_corrected']
# plt.annotate('actual group', xy=(x+0.2,y), xytext=(x+0.3, 300),
#              arrowprops=dict(facecolor='black', shrink=0.05, headwidth=20, width=7))



# MSIT single model vs INC betas

In [ ]:
betas_inc = load_betas('inc').drop(['event type', 'covariate', 'regressor'], axis=1)
betas_con = load_betas('con').drop(['event type', 'covariate', 'regressor'], axis=1)
cog = load_single_msit()

cog.subject = cog['Unnamed: 0'].astype(str)
cog= cog.drop('Unnamed: 0', axis=1)

# subs = [x for x in betas.subject if any(s in x for s in cog.subject)] 
# all_subs =[x for x in subs if any(s in x for s in betas.subject)] 
all_subs = sorted(list(set(betas_inc.subject).intersection(cog.subject)),key=int)
cog = cog.loc[cog.subject.isin(all_subs)].reset_index(drop=True)
betas_inc = betas_inc.loc[betas_inc.subject.isin(all_subs)].reset_index(drop=True)

cog_betas = pd.concat([cog, betas_inc], axis=1)
cog_betas = cog_betas.loc[:,~cog_betas.columns.duplicated()]

betas_con = betas_con.loc[betas_con.subject.isin(all_subs)].reset_index(drop=True)

inc_con_betas = betas_inc.set_index('subject') - betas_con.set_index('subject')
inc_con_betas = inc_con_betas.reset_index()
cog_inc_con_betas = pd.concat([cog, inc_con_betas], axis=1)
cog_inc_con_betas = cog_inc_con_betas.loc[:,~cog_inc_con_betas.columns.duplicated()]

cog_inc_con_betas['vIncongruent'] = cog_inc_con_betas['vSimon'] + cog_inc_con_betas['vFlank'] #+ cog_inc_con_betas['vMatch']# + cog_inc_con_betas['v']

In [ ]:
print(len(all_subs)) # there should be 36

In [ ]:
## correct for multiple comparisons

# pick regions we expect correlations with
# dmc_betas = 
cog_inc_con_betas_cor = cog_inc_con_betas[['subject','ACC-l','ACC-r','IFG-l','IFG-r','CN-l','CN-r','vIncongruent']] # 'Tha-l','Tha-r''Ins-l','Ins-r',

cog_inc_con_betas_cor = cog_inc_con_betas_cor.set_index('subject')
cog_inc_con_betas_cor = cog_inc_con_betas_cor.melt(var_name='ROI',value_name='beta',ignore_index=False)

tmp = cog_inc_con_betas_cor.groupby('ROI')['beta'].apply(lambda x: scipy.stats.pearsonr(x, cog_inc_con_betas_cor.loc[cog_inc_con_betas_cor.ROI=='vIncongruent','beta']))
stat_df = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['t', 'p'])
stat_df = pd.concat([stat_df, pd.DataFrame(statsmodels.stats.multitest.fdrcorrection(stat_df['p'], method='i'), index=['fdr_significant', 'p_corrected'], columns=stat_df.index).T], axis=1)

stat_df['p_corrected'] = stat_df['p_corrected'].astype(float).round(3)
stat_df.loc[stat_df.index=='ACC-r','p_corrected'] = ' 0.003'

stat_df

In [ ]:
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True,figsize=(28,6))
# fig.suptitle('Correlations between cognitive estimates and betas')
ax = axes.ravel()
rois = ['ACC','IFG','CN'] # 'Ins',,'Tha'
hemis = ['l','r']
for i, r in enumerate(rois):
    for h in hemis:
        this_roi = r+'-'+h
        coloring = 'darkblue' if h == 'l' else 'cornflowerblue'
        sns.regplot(x=cog_inc_con_betas['vIncongruent'], y=cog_inc_con_betas[this_roi],ax=ax[i], truncate=False, ci=95, color=coloring)
        ax[i].set_title(r,fontsize=26)
        if i in [0,3]:
            ax[i].set_ylabel('beta (INC - CON)', fontsize=24)
        else:
    #         ax[i].set_xlabel('')
            ax[i].set_ylabel('')
        ax[i].set_xlabel('v (INC - CON)', fontsize=22)
        ax[i].tick_params(axis='both', which='major', labelsize=22)
        if '-l' in this_roi:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_inc_con_betas["vIncongruent"], cog_inc_con_betas[this_roi])[0].round(3)}', (1,0.195),fontsize=18, color='darkblue')    
        else:
            ax[i].annotate(f'p={stat_df.loc[stat_df.index == this_roi,"p_corrected"].to_string(index=False)}\nr={pearsonr(cog_inc_con_betas["vIncongruent"], cog_inc_con_betas[this_roi])[0].round(3)}', (1,0.146),fontsize=18, color='cornflowerblue')    

# ax[5].axis('off')
#     ax[11].axis('off')

fig.savefig('figure_download_scott/Figure15_new.pdf', bbox_inches='tight')

# correlate dmc sst params with mpwg msit params

In [ ]:
betas = load_betas('ss') 
dmc = pd.read_csv('scott/dmc_params_final.tsv', sep='\t') 
dmc['subject'] = dmc.index
dmc.drop(dmc.tail(1).index,inplace=True) # drop last n rows dmc.reset_index() 
dmc['SSRT'] = dmc['muS'] + dmc['tauS'] 

In [ ]:
subs_to_keep = [x for x in list(dmc.subject) if '25' not in x]
subs_to_keep = [x for x in list(subs_to_keep) if '35' not in x]
subs_to_keep = [x for x in list(subs_to_keep) if '41' not in x]

In [ ]:
cog_all = cog.loc[cog['subject'].isin(subs_to_keep)].reset_index(drop=True)
dmc_all = dmc.loc[dmc['subject'].isin(subs_to_keep)].reset_index(drop=True)
dmc_all = dmc_all.drop('subject',axis=1)

In [ ]:
all_params = pd.concat([cog_all,dmc_all],axis=1)
all_params = all_params.drop('subject',axis=1)

In [ ]:
all_params

# full correlation heatmaps

In [ ]:
# Increase the size of the heatmap.
plt.figure(figsize=(16, 6))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(all_params.corr(), vmin=-1, vmax=1, annot=True)


In [ ]:
import scipy
plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
# mask = np.triu(np.ones_like(all_params.corr(), dtype=np.bool))
def corr_sig(df=None):
    p_matrix = np.zeros(shape=(df.shape[1],df.shape[1]))
    for col in df.columns:
        for col2 in df.drop(col,axis=1).columns:
            _ , p = scipy.stats.pearsonr(df[col],df[col2])
            p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = p
    return p_matrix

p_values = corr_sig(all_params)
mask = np.invert(np.tril(p_values<0.05))

heatmap = sns.heatmap(all_params.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Model Parameter Correlations', fontdict={'fontsize':18}, pad=16);

In [ ]:
plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(all_params.corr(), dtype=np.bool))
heatmap = sns.heatmap(all_params.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Model Parameter Correlations', fontdict={'fontsize':18}, pad=16);

### do correlation with pnormed gf and tf values

In [ ]:
adj_params = all_params.copy()
adj_params['tf'] = scipy.stats.norm.cdf(adj_params['tf'])
adj_params['gf'] = scipy.stats.norm.cdf(adj_params['gf'])

In [ ]:
adj_params.rename(columns={'vFlank':'$\u03BD_{Flank}$','vSimon':'$\u03BD_{Simon}$','vPos.1':'$\u03BD_{pos1}$','vPos.2':'$\u03BD_{pos2}$','vMatch':'$\u03BD_{Target}$','v':'$\u03BD_{0}$',
                           'mu.true':'$\u03BC_{go}$','sigma.true':'$\u03C3_{go}$','tau.true':'$\u03C4_{go}$',
                           'muS':'$\u03BC_{stop}$','sigmaS':'$\u03C3_{stop}$','tauS':'$\u03C4_{stop}$'},inplace=True)

In [ ]:
plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(adj_params.corr(), dtype=np.bool))
heatmap = sns.heatmap(adj_params.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Model Parameter Correlations', fontdict={'fontsize':18}, pad=16);

In [ ]:
import scipy
plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
# mask = np.triu(np.ones_like(all_params.corr(), dtype=np.bool))
def corr_sig(df=None):
    p_matrix = np.zeros(shape=(df.shape[1],df.shape[1]))
    for col in df.columns:
        for col2 in df.drop(col,axis=1).columns:
            _ , p = scipy.stats.pearsonr(df[col],df[col2])
            p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = p
    return p_matrix

p_values = corr_sig(adj_params)
mask = np.invert(np.tril(p_values<0.05))

heatmap = sns.heatmap(adj_params.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Model Parameter Correlations', fontdict={'fontsize':18}, pad=16);

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib.patches as mpatches
# get the p value for pearson coefficient, subtract 1 on the diagonal
pvals = adj_params.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*adj_params.corr().shape)
# set the significance threshold
psig = 0.05

plt.figure(figsize=(26,6))

mask = np.triu(np.ones_like(adj_params.corr(), dtype=np.bool))

heaty = sns.heatmap(adj_params.corr()[pvals<psig], mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
sns.heatmap(adj_params.corr()[pvals>=psig], mask=mask, vmin=-1, vmax=1, annot=True, cmap=sns.color_palette(['lightgrey']))
heaty.set_title('Model Parameter Correlations', fontdict={'fontsize':18}, pad=16);

# colors = [sns.color_palette("Greys", n_colors=1, desat=1)[0]]
# texts = [f"n.s. (at {psig})"]
# patches = [ mpatches.Patch(color=colors[i], label="{:s}".format(texts[i]) ) for i in range(len(texts)) ]
# plt.legend(handles=patches, bbox_to_anchor=(.85, 1.05), loc='center')
# sns.heatmap(adj_params.corr()[pvals<psig], annot=True, square=True)

# # add another heatmap with colouring the non-significant cells
# sns.heatmap(adj_params.corr()[pvals>=psig], annot=True, square=True, cbar=False, 
#             cmap=sns.color_palette("Greys", n_colors=1, desat=1))



In [ ]:
import pingouin

In [ ]:
# adjust everything for fdr
fdr_cor = pingouin.rcorr(adj_params, method='pearson', upper='pval', decimals=3, padjust='fdr_bh', stars=False).replace('-',1).apply(pd.to_numeric)
pvals = fdr_cor.transpose()
psig = 0.05

plt.figure(figsize=(26,6))

mask = np.triu(np.ones_like(adj_params.corr(), dtype=np.bool))
# sns.set(font_scale=1.3) 
heaty = sns.heatmap(fdr_cor[pvals<psig], mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG',annot_kws={"size": 15})
sns.heatmap(fdr_cor[pvals>=psig], mask=mask, vmin=-1, vmax=1, annot=True, cmap=sns.color_palette(['lightgrey']), annot_kws={"size": 14})
heaty.set_title('Model Parameter Correlations', fontdict={'fontsize':18}, pad=16);
heaty.tick_params(labelsize=15)
cbar = heaty.collections[0].colorbar
# here set the labelsize by 20
cbar.ax.tick_params(labelsize=16)

